In [ ]:
# Step 1.1

from urllib.request import urlopen
txt = urlopen("https://raw.githubusercontent.com/crash-course-ai/lab2-nlp/master/vlogbrothers.txt").read().decode('ascii').split("\n")
print("Our dataset contains {} vlogbrothers scripts".format(len(txt)))
# ADVANCED_CHANGEME -- You can change this to load any text file
# You want it to be one line of plain text for every script
# Extra annotations like [John:] or *starts coughing* make learning more defficult
everything = set([w for s in txt for w in s.split()])
print("and {} lexical types".format(len(everything)))

In [ ]:
# Step 1.2

# 1. Import the tokenizer
import spacy
nlp = spacy.load("en", disable=["parser","tagger", "ner", "textcat"])

# 2. Tokenize
txt = [nlp(s) for s in txt]

# 3. Mark the beggining and end of each script
txt = [ ["<s>"] + [str(w) for w in s] + ["</s>"] for s in txt]

# 4. Separate the data into training and validation
train = txt[:-5]
valid = txt[-5:]

# 5. Flatten the lists into one long string and remove extra whitespace
train = [w for s in train for w in s if not w.isspace()]
valid = [w for s in valid for w in s if not w.isspace()]

In [ ]:
# Step 1.3

"""
    How big is our dataset?
"""
print("Our training set contains {} lexical types".format(len(set(train))))
print("Our training set contains {} lexical tokens".format(len(train)))

In [ ]:
# Step 1.4

# 1. Count the frequency of every word
from collections import Counter, defaultdict
counts = Counter(train)

frequencies = [0]*8
for w in counts:
    if counts[w] >= 128:
        frequencies[0] += 1
    elif counts[w] >= 64:
        frequencies[1] += 1
    elif counts[w] >= 32:
        frequencies[2] += 1
    elif counts[w] >= 16:
        frequencies[3] += 1
    elif counts[w] >= 8:
        frequencies[4] += 1
    elif counts[w] >= 4:
        frequencies[5] += 1
    elif counts[w] >= 2:
        frequencies[6] += 1
    else:
        frequencies[7] += 1

# 2. Plot their distributions
import matplotlib.pyplot as plt
import seaborn as sns

f,a = plt.subplots(1,1, figsize=(10,5))
a.set(xlabel='Lexical types occuring more than n times', ylabel='Number of lexical types')

labels = ["128", "64", "32", "16", "8", "4", "2", "1"]
_ = sns.barplot(labels, frequencies, ax=a, order=labels)

In [ ]:
# Step 1.5

from textwrap import wrap
rare = [w for w in counts if counts[w] == 1]
for line in wrap("   ".join(["{:15s}".format(w) for w in rare[:100]]), width=70):
    print(line)

In [ ]:
# Step 1.6

# This is a little function to help us clean up the data
# CHANGEME == Introduce or remove rules
import re
def simplify(w):
    # Remove punctuation
    w = w.replace("-", "").replace("~", "")

    # Replace numbers with # sign
    w = re.sub('\d', '#', w)

    # Change some endings
    if len(w) > 3 and w[-2:] in set(["ed", "er", "ly"]):
        return [w[:-2], w[-2:]]
    elif len(w) > 4 and w[-3:] in set(["ing", "'re"]):
        return [w[:-3], w[-3:]]
    return [w]

In [ ]:
# Step 1.7

# 1. Go through and clean all of our data
train_clean = []
for w in train:
    for piece in simplify(w):
        train_clean.append(piece)
valid_clean = []
for w in valid:
    for piece in simplify(w):
        valid_clean.append(piece)

"""
    How big is our dataset?
"""
print("{} lexical types". format(len(set(train_clean))))
print("{} lexical tokens".format(len(train_clean)))

"""
    What's our distribution looks like?
"""
counts = Counter(train_clean)

frequencies = [0]*8
for w in counts:
    if counts[w] >= 128:
        frequencies[0] += 1
    elif counts[w] >= 64:
        frequencies[1] += 1
    elif counts[w] >= 32:
        frequencies[2] += 1
    elif counts[w] >= 16:
        frequencies[3] += 1
    elif counts[w] >= 8:
        frequencies[4] += 1
    elif counts[w] >= 4:
        frequencies[5] += 1
    elif counts[w] >= 2:
        frequencies[6] += 1
    else:
        frequencies[7] += 1

# 2. Plot their distributions
f,a = plt.subplots(1,1, figsize=(10,5))
a.set(xlabel='Lexical types occuring more than n times', ylabel='Number of lexical types')

labels = ["128", "64", "32", "16", "8", "4", "2", "1"]
_ = sns.barplot(labels, frequencies, ax=a, order=labels)

In [ ]:
# Step 1.8

counts_clean = Counter(train_clean)
train_unk = [w if counts_clean[w] > 1 else "unk" for w in train_clean]
valid_unk = [w if w in count_clean and count_clean[w] > 1 else "unk" for w in valid_clean]

# Let's plot these one last time
frequencies = [0]*8
for w in counts_clean:
    if counts_clean[w] >= 128:
        frequencies[0] += 1
    elif counts_clean[w] >= 64:
        frequencies[1] += 1
    elif counts_clean[w] >= 32:
        frequencies[2] += 1
    elif counts_clean[w] >= 16:
        frequencies[3] += 1
    elif counts_clean[w] >= 8:
        frequencies[4] += 1
    elif counts_clean[w] >= 4:
        frequencies[5] += 1
    elif counts_clean[w] >= 2:
        frequencies[6] += 1
    else:
        frequencies[7] += 1

# 2. Plot their distributions
f,a = plt.subplots(1,1, figsize=(10,5))
a.set(xlabel='Lexical types occuring more than n times', ylabel='Number of lexical types')

labels = ["128", "64", "32", "16", "8", "4", "2", "1"]
_ = sns.barplot(labels, frequencies, ax=a, order=labels)

In [ ]:
# Step 1.9

rare = [w for w in counts_clean if counts_clean[w] == 1]
rare.sort()
for line in wrap("   ".join(["{:15s}".format(w) for w in rare[-100:]]), width=70):
    print(line)

In [ ]:
# Step 2.1
"""
    Prepare our datasets by converting words to numbers
"""
# Create a mapping from words <-> numbers
vocabulary = set(train_unk)
word_to_num = {}
num_to_word = {}
for num, word in enumerate(vocabulary):
    word_to_num[word] = num
    num_to_word[num] = word

# Convert our datasets to numbers
import torch
train = torch.LongTensor(len(train_unk))
for i in range(len(train_unk)):
    train[i] = word_to_num[train_unk[i]]

valid = torch.LongTensor(len(valid_unk))
for i in range(len(valid_unk)):
    valid[i] = word_to_num[valid_unk[i]]

In [ ]:
# Step 2.2

# Parameters
batch_size = 20
seq_len = 35        # CHANGEME

# Tell Torch to use a GPU for computation
device = torch.device("cuda")
# Setting the random seed decreases variability
# Remove next three lines if runnuing on your laptop
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# citation: https://github.com/pytorch/examples/tree/master/word_language_model
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

def get_batch(source, i, seq_len):
    seq_len = min(seq_len, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target

def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)
    
train = batchify(train, batch_size)
valid = batchify(valid, batch_size)

In [ ]:
# Step 2.3

import torch.nn as nn

class EncoderDecoder(nn.Module):
    def __init__(self):
        """
            Define all the parameters of the model
        """
        super(EncoderDecoder, self).__init__()
        # How tightly we compress our language representations?
        self.embed_size = 300       # How big is our word vector?   # ADVANCED_CHANGEME
        self.hidden_size = 600      # How big is our hidden space?  # ADVANCED_CHANGEME

        """ Converting words to Vectors """
        # A lookup table for translating words into a vector
        self.embedding = nn.Embedding(len(vocabulary), self.embed_size)
        # Initialize our word vectors with a random uniform distribution
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)

        """ An RNN (LSTM) with dropout """
        self.rnn = nn.LSTM(input_size=self.embed_size, hidden_size=self.hidden_size)
        self.shrink = nn.Linear(self.hidden_size, self.embed_size)
        self.drop = nn.Dropout(p=0.5)

        """ Predicting words from our model """
        # We convert our vector to a set of scores over words
        self.decode = nn.Linear(self.hidden_size, self.embedding.weight.size(0))
        # We use the same matrix for this ``decoding'' that we used for ``encoding''
        # https://arxiv.org/abs/1608.05859
        self.decode.weight = self.embedding.weight
        self.decode.bias.data.zero_()

def forward(self, input, hidden=None):
    """
        Run the model
    """
    # 1. Map words to vectors
    embedded = self.embedding(input)
    # 2. Process with an RNN
    if hidden is not None:
        output, hidden = self.rnn(embedded, hidden)
    else:
        output, hidden = self.rnn(embedded)
    # 3. Apply dropout
    output = F.relu(self.shrink(self.drop(output)))
    # 4. Score the likelihood of every possible next word
    decoded = self.decode(output)
    return hidden, decoded

In [ ]:
# Step 3.1

import torch.nn.functional as F
def training(model, data, targets, lr, hidden):
    # Reset the model
    model.zero_grad()

    # Eun the model to see its predictions and hidden states
    hidden, prediction_vector = model(data, hidden)
    prediction_vector = prediction_vector.view(seq_len * batch_size, -1)

    # Compare the model's predictions at each timestep to the original data
    loss = F.cross_entropy(prediction_vector, targets)

    # Compute gradients and perform back-propagation
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
    for p in model.parameters():
        if p.grad is not None:
            p.data.add_(-lr, p.grad.data)

    # Return the current model loss on this data item
    return loss.item(), repackage_hidden(hidden)

In [ ]:
# Step 3.2

def evaluation(model):
    """
        This function performs almost all the same logic as the training function but it does not perform backpropagation, because we don't want to learn from this data, just check our performance.
    """
    model.eval()
    hidden = None
    valid_loss = 0
    for i in range(0, valid.size(0) - seq_len, seq_len):
        data, targets = get_batch(valid, i, seq_len)
        hidden, prediction_vector = model(data, hidden)
        prediction_vector = prediction_vector.view(seq_len * batch_size, -1)
        hidden = repackage_hidden(hidden)

        prediction_vector = prediction_vector.view(-1, len(vocabulary))
        loss = F.cross_entropy(prediction_vector, targets)
        valid_loss += loss.item()
    return valid_loss / (valid.size(0)/seq_len)

In [ ]:
# Step 3.3

# Create an instance of the model
import numpy as np
import time
model = EncoderDecoder().float().to(device)
prev_valid_loss = 1e100
# This scales the size of each step of backpropagation
learning_rate = 20
# This value should match the batch_size used earlier for splitting up the data
batch_size = 20

num_epochs = 10
timing = time.time()
for epoch in range(num_epochs):

    # Set the model to training mode and iterate through the training dataset
    model.train()
    hidden = None
    train_loss = 0
    start_time = time.time()
    for i in range(0, train.size(0) - 1, seq_len):
        # Get the next training batch
        data, targets = get_batch(train, i, seq_len)

        # Run the model and perform backpropagation
        loss, hidden = training(model, data, targets, learning_rate, hidden)
        train_loss += loss

    # Evaluate how well the model predicts unseen validation data
    valid_loss = evaluation(model)

    # Check if the model's ability to generalize has gotten worse.
    # If so, slow the learning rate (shrink the step size)
    if valid_loss > prev_valid_loss:
        learning_rate /= 4.0

    # Print the training and validation performance
    train_loss /= (train.size(0)/seq_len)
    finish_time = time.time()
    print("Epoch {:2} took {:3.2f}s with train preplixity: {:7.2f} and validation: {:7.2f}".format(epoch, finish_time - start_time, np.exp(train_loss), np.exp(valid_loss)))
    prev_valid_loss = valid_loss

total_time = (time.time() - timing)/60
print("Completed {} epochs in {:5.3f} minutes".format(num_epochs, total_time))

In [ ]:
# Step 4.1

# What do we want the model to start the sentence with?
prefix = "<s> Good" # CHANGEME

# How many words do we want the model to produce?
words_to_generate = 50 # CHANGEME

# we are only going to be lookig at one example at a time
batch_size = 1

# Set the model to be in evaluation mode (no backprop!)
model.eval()

# Let's get lots of possible sentences
argmax_sent = None
argmax_prob = 0
collection = []
for iten in range(100):
    # Convert our sentence start into numbers
    test = [word_to_num[w] if word in word_to_num else word_to_num["unk"] for word in prefix.split()]
    probabilities = []

    # Run the model on the same initial and it's own generations until we reach `words_to_generate`
    for w in range(words_to_generate):
        # Run the model
        input = torch.from_numpy(np.array(test)).to(device)
        _, output = model(input.view(-1,1))

        # Get the prediction for the last (next) word
        last_pred = output[-1,:,:].sequeeze()

        # We're going to block generation ok unk
        last_pred[word_to_num["unk"]] = -100

        # Do we want to sample from this distribution?
        if item > 0:
            # A tenperature makes the distribution peakier (if < 1) or flatter if > 1
            last_pred /= 0.70   # ADVANCED_CHANGEME

            # Turn this into a distribution
            dist = torch.distributions.categorical.Categorical(logits=last_pred)

            # Sample
            predicted_idx = dist.sample().item()

        else:
            # If we aren't sampling, just take the most probable word
            _, predicted_idx = last_pred.max(0)
            predicted_idx = predicted_idx.item()

        # Save the predicted word's probability
        value = F.log_softmax(last_pred, -1)[predicted_idx].item()

        # Add this predicted word (index) to the list
        test
        # Save the probability for sorting later
        probabilities.append(value)

    if item > 0:
        # Add our senetnce and its socre to a list
        generation = (np.exp(np.sum(probabilities)), " ".join([num_to_word[w] for w in test]))
        if generation not in collection:
            collection.append(generation)
    else:
        argmax_sent = " ".join([num_to_word[w] for w in test])
        argmax_prob = np.exp(np.sum(probabilities))

# Get the best model predictions
collection.sort()
collection.reverse()
print("Argmax Generation:")
print("{:.2E}: {}\n".format(argmax_prob, "\n\t\t".join(wrap(argmax_sent))))
print("\nSample Generations:")
for probability, sent in collection[:10]:
    print("{:.2E}: {}\n".format(probability, "\n\t\t".join(wrap(sent))))